In [18]:
# Loading libraries

import os
from dotenv import load_dotenv
from huggingface_hub import login
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import pandas as pd
from rapidfuzz import fuzz
from codecarbon import EmissionsTracker
from sklearn.model_selection import train_test_split
from utils import *
import optuna
from optuna.samplers import TPESampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sentence_transformers import SentenceTransformer

random_state = 42

In [19]:
# Load variables from .env
load_dotenv()

# Get token from .env
hf_token = os.getenv("HF_TOKEN")

# Log in to HF Hub
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [20]:
# Define the path to the Parquet dataset
path = "dataset/leandl_oesnpg_dados.parquet"

# Load the Parquet file into a Pandas DataFrame
df = pd.read_parquet(path)

# Display the DataFrame
df

,hash_id,tema_id,tema,palavras_chave,uf_tema_info,uf_pesquisador,nome_programa,sigla_entidade_ensino,nome_producao,nome_subtipo_producao,...,descricao_abstract,descricao_keyword,data_titulacao,nome_grau_academico,nome_grande_area_conhecimento,nome_area_conhecimento,nome_subarea_conhecimento,nome_especialidade,modelo_nivel,modelo_explicacao
0,ce4025a58d1cff3d346e96af2e8f2d0185caeaddd78c1b...,1,Agronegócio e Tecnologias de Informação e Comu...,"[agroindústria, assistência técnica e extensão...",ACRE,ACRE,ENSINO DE CIÊNCIAS E MATEMÁTICA,UFAC,AS TECNOLOGIAS DIGITAIS DA INFORMAÇÃO E COMUNI...,DISSERTAÇÃO,...,"THE RESEARCH, DESCRIBED IN THE PRESENT WORK, I...",INFORMATION AND COMMUNICATION TECHNOLOGIES;COM...,2023-10-18 00:00:00,MESTRADO PROFISSIONAL,MULTIDISCIPLINAR,ENSINO,ENSINO DE CIÊNCIAS E MATEMÁTICA,NÃO SE APLICA,BAIXA,A afinidade entre os dados do pesquisador e o ...
1,55982d77d62446fb9f76ae636c99c36d77d5e233ce4863...,1,Agronegócio e Tecnologias de Informação e Comu...,"[agroindústria, assistência técnica e extensão...",ACRE,ACRE,EDUCAÇÃO PROFISSIONAL E TECNOLÓGICA,IFAC,O CURRÍCULO INTEGRADO DO INSTITUTO FEDERAL DO ...,DISSERTAÇÃO,...,THIS STUDY INVESTIGATED THE CURRICULUM OF INTE...,INTEGRATED SECONDARY EDUCATIO;CURRICULAR ORGAN...,2023-09-29 00:00:00,MESTRADO PROFISSIONAL,MULTIDISCIPLINAR,ENSINO,NÃO SE APLICA,NÃO SE APLICA,MEDIA,A afinidade entre os dados do pesquisador e o ...
2,1f7615b9be49f80d289ba7c99eb64a5f8dc387a23518a6...,3,Biodiversidade e Biotecnologia,"[biodiversidade, bioeconomia, biotecnologia, c...",ACRE,ACRE,CIÊNCIAS DA SAÚDE NA AMAZÔNIA OCIDENTAL,UFAC,TENDÊNCIA TEMPORAL E DISTRIBUIÇÃO ESPACIAL DAS...,DISSERTAÇÃO,...,"THE ANALYZES IN THE BIOMES: AMAZON, CAATINGA, ...",EPIDEMIOLOGY;LEISHMANIA;PRAIS-WINSTEN,2023-03-21 00:00:00,MESTRADO,CIÊNCIAS DA SAÚDE,MEDICINA,ANATOMIA PATOLÓGICA E PATOLOGIA CLÍNICA,NÃO SE APLICA,BAIXA,Os dados do pesquisador(a) estão focados na an...
3,3773fcb7084d294753146c9580750eca9b2348b78cc4aa...,3,Biodiversidade e Biotecnologia,"[biodiversidade, bioeconomia, biotecnologia, c...",ACRE,ACRE,GEOGRAFIA,UFAC,MODELAGEM DE BIOMASSA FLORESTAL E CÁLCULO DE C...,DISSERTAÇÃO,...,DURING THE LAST FEW DECADES TROPICAL FORESTS H...,GEDI;REMOTE SENSING;FOREST;MAPPING;BIOMASS;CARBON,2023-08-25 00:00:00,MESTRADO,CIÊNCIAS HUMANAS,GEOGRAFIA,NÃO SE APLICA,NÃO SE APLICA,MEDIA,"A dissertação do pesquisador foca na ""modelage..."
4,07dae75eae08bbf1828e779c70c9d283965fe58880d02f...,3,Biodiversidade e Biotecnologia,"[biodiversidade, bioeconomia, biotecnologia, c...",ACRE,ACRE,CIÊNCIAS AMBIENTAIS,UFAC,ANÁLISE SOCIOECONÔMICA E AMBIENTAL DA CADEIA P...,DISSERTAÇÃO,...,THE AMAZON BIOME HOLDS A PROMINENT POSITION IN...,BURITI;NON-TIMBER FOREST PRODUCTS;COOPERATIVIS...,2023-01-30 00:00:00,MESTRADO,MULTIDISCIPLINAR,CIÊNCIAS AMBIENTAIS,NÃO SE APLICA,NÃO SE APLICA,ALTA,Os dados do pesquisador mostram uma clara afin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42041,34aa7153bcaefa1132d1117f40f41f8e46781282b23d2f...,467,Transformação Digital,"[automação de processos, cibersegurança, desbu...",TOCANTINS,TOCANTINS,GOVERNANÇA E TRANSFORMAÇÃO DIGITAL,UFT-PALMAS,COMAIS LABS: IMPLANTAÇÃO E TRANSFORMAÇÃO DIGIT...,TESE,...,THIS WORK ADDRESSES THE IMPLEMENTATION AND DIG...,DIGITAL TRANSFORMATION;SOLUTION MODELING;ARTIF...,2023-07-21 00:00:00,DOUTORADO PROFISSIONAL,MULTIDISCIPLINAR,INTERDISCIPLINAR,ENGENHARIA/TECNOLOGIA/GESTÃO,NÃO SE APLICA,ALTA,O título da tese 'COMAIS LABS: IMPLANTAÇÃO E T...
42042,993855663c31514350f2f5d9926141bf24b534bc3486a8...,467,Transformação Digital,"[automação de processos, cibersegurança, desbu...",TOCANTINS,TOCANTINS,PROFNIT - PROPRIEDADE INTELECTUAL E TRANSFERÊN...,UFT-PALMAS,DESENVOLVIMENTO DE JOGO EDUCATIVO SOBRE PROPRI...,DISSERTAÇÃO,...,IT IS A PROTOTYPE FOR THE DESIGN OF A BOARD GA...,BOARD GAME;INTELECTUAL PROPERTY;TECHNOLOGY TRA...,2023-04-04 00:00:00,MESTRADO PROFISSIONAL,CIÊNCIAS SOCIAIS APLICADAS,ADMINISTRAÇÃO,NÃO SE APLICA,NÃO SE APLICA,BAIXA,O te

In [21]:
# Creating the "afinidade" column
df["afinidade"] = df["modelo_nivel"].apply(
    # Assign 0 if the value in "modelo_nivel" is either "BAIXA" or "MEDIA"
    # Otherwise, assign 1
    lambda x: 0 if x in ["BAIXA", "MEDIA"] else 1
)

In [22]:
# Creating the "perfil" column by concatenating two text fields
df['perfil'] = (
    df['nome_producao'].fillna("")   # Replace NaN with an empty string in 'nome_producao'
    + "\n"                           # Add a line break between the two fields
    + df["descricao_resumo"].fillna("")  # Replace NaN with an empty string in 'descricao_resumo'
)

# Display the "perfil" column
df['perfil']

0        AS TECNOLOGIAS DIGITAIS DA INFORMAÇÃO E COMUNI...
1        O CURRÍCULO INTEGRADO DO INSTITUTO FEDERAL DO ...
2        TENDÊNCIA TEMPORAL E DISTRIBUIÇÃO ESPACIAL DAS...
3        MODELAGEM DE BIOMASSA FLORESTAL E CÁLCULO DE C...
4        ANÁLISE SOCIOECONÔMICA E AMBIENTAL DA CADEIA P...
                               ...                        
42041    COMAIS LABS: IMPLANTAÇÃO E TRANSFORMAÇÃO DIGIT...
42042    DESENVOLVIMENTO DE JOGO EDUCATIVO SOBRE PROPRI...
42043    DESAFIOS E OPORTUNIDADES DA IMPLEMENTAÇÃO DA I...
42044    ECOSSISTEMAS NA COMUNIDADE ACADÊMICA E EMPRESA...
42045    TURISMO SUSTENTÁVEL EM PRAIA DE RIO: PERCEPÇÃO...
Name: perfil, Length: 42046, dtype: object

In [23]:
# First: split into train (70%) and temp (30%)
train, temp = train_test_split(
    df,
    test_size=0.30,
    stratify=df['afinidade'],   # preserve class distribution in "afinidade"
    random_state=random_state
)

# Then: split the remaining temp into validation (10%) and test (20%)
# Since temp contains 30% of the data, we need to keep the right proportions:
# val = 10/30 = 0.3333 of temp
val, test = train_test_split(
    temp,
    test_size=2/3,  # equivalent to 20% of the total (2/3 of temp)
    stratify=temp['afinidade'],
    random_state=random_state
)

print(f"Train: {len(train)}")
print(f"Val: {len(val)}")
print(f"Test: {len(test)}")


Train: 29432
Val: 4204
Test: 8410


# String Match

In [7]:
# Define the path for saving the string match results
path_string_match = "results/string_match"

# Create the directory if it does not exist
create_directory(path_string_match)

Directory 'results/string_match' already exists.


In [ ]:
# Start CodeCarbon tracker to measure carbon emissions during execution
tracker = EmissionsTracker(output_dir=path_string_match, save_to_file=True, project_name="test")
tracker.start()

# Define a simple string matching function:
# Returns 1 if the value of 'tema' is found inside 'perfil', otherwise 0
def string_match(row):
    return int(row['tema'].lower() in row['perfil'].lower())

# Apply the string_match function row by row to the test set
test['pred_string'] = test.apply(string_match, axis=1)

# Generate classification report comparing true labels ('afinidade') 
# against predictions ('pred_string'), and return it as a dictionary
report = classification_report(test['afinidade'], test['pred_string'], output_dict=True)

# Stop the CodeCarbon tracker and store estimated emissions
emissions = tracker.stop()

# Save the classification report to a JSON file
save_dict_to_json(report, os.path.join(path_string_match, "test_classification_report.json"))

print("✅ Results saved in", path_string_match)
print("Estimated emissions:", emissions, "kg CO₂")

[codecarbon WARNING @ 17:34:20] Multiple instances of codecarbon are allowed to run at the same time.


✅ Dictionary saved at: results/string_match/classification_report.json
✅ Results saved in results/string_match
Estimated emissions: 5.344569361430485e-07 kg CO₂


# Fuzzy Match

In [9]:
# Define the path for saving the fuzzy match results
path_fuzzy_match = "results/fuzzy_match"

# Create the directory if it does not exist
create_directory(path_fuzzy_match)

Directory 'results/fuzzy_match' already exists.


In [10]:
path_fuzzy_match = "results/fuzzy_match"

# Fuzzy prediction function
# Returns 1 if the similarity between 'tema' and 'perfil' is above the threshold, otherwise 0
def fuzzy_predict(df, threshold):
    return df.apply(
        lambda row: int(
            fuzz.partial_ratio(row['tema'].lower(), row['perfil'].lower()) >= threshold
        ),
        axis=1
    )

# Combine training and validation sets for threshold tuning
train_val = pd.concat([train, val])

best_threshold = None
best_f1 = 0
results_train = {}

# -------------------------
# 🚀 Measuring carbon emissions during threshold search
# -------------------------
tracker_train = EmissionsTracker(
    output_dir=path_fuzzy_match,
    save_to_file=True,
    project_name="train_val"
)
tracker_train.start()

# Loop over possible thresholds (5, 10, ..., 100)
for t in range(5, 101, 5): 
    preds = fuzzy_predict(train_val, threshold=t)
    f1 = f1_score(train_val['afinidade'], preds)
    results_train[t] = {"f1_score": f1}
    # Update best threshold if a higher F1-score is found
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

# Stop emissions tracker and log results
emissions_train = tracker_train.stop()
print(f"🌱 Total carbon emissions (train/validation): {emissions_train:.6f} kg CO₂")

# Save F1-scores for all thresholds and the best one
save_dict_to_json(results_train, f"{path_fuzzy_match}/train_val_f1.json")
save_dict_to_json(
    {"best_threshold": best_threshold, "best_f1": best_f1, "emissions_train": emissions_train},
    f"{path_fuzzy_match}/best_threshold.json"
)

print(f"Best threshold on train+val: {best_threshold} (F1={best_f1:.3f})")

# -------------------------
# 🚀 Measuring carbon emissions on the test set with the best threshold
# -------------------------
tracker_test = EmissionsTracker(
    output_dir=path_fuzzy_match,
    save_to_file=True,
    project_name="test"
)
tracker_test.start()

# Apply the chosen threshold to the test set
test_preds = fuzzy_predict(test, threshold=best_threshold)
report_test = classification_report(test['afinidade'], test_preds, output_dict=True)

# Stop tracker and store emissions
emissions_test = tracker_test.stop()
print(f"🌱 Carbon emissions (test): {emissions_test:.6f} kg CO₂")

# Save test report
save_dict_to_json(report_test, f"{path_fuzzy_match}/test_classification_report.json")

print("✅ Results and emissions saved in", path_fuzzy_match)

🌱 Total carbon emissions (train/validation): 0.000022 kg CO₂
✅ Dictionary saved at: results/fuzzy_match/train_val_f1.json
✅ Dictionary saved at: results/fuzzy_match/best_threshold.json
Best threshold on train+val: 55 (F1=0.385)
🌱 Carbon emissions (test): 0.000001 kg CO₂
✅ Dictionary saved at: results/fuzzy_match/test_classification_report.json
✅ Results and emissions saved in results/fuzzy_match


# BERT

In [24]:
def compute_and_save_embeddings(embedding_model, train, val, test,
                                profile_col="perfil", theme_col="tema", label_col="afinidade"):
    """
    Generate embeddings for train/val/test and measure carbon emissions separately:
      - Training + Validation
      - Test
    Only computes if the files do not already exist.
    """
    print(f"\n🟢 Checking embeddings for {embedding_model}...")
    embed_path = f"embeddings/{embedding_model.replace('/', '_')}"
    create_directory(embed_path)

    files_needed = [
        f"{embed_path}/X_profile_train.npy",
        f"{embed_path}/X_theme_train.npy",
        f"{embed_path}/y_train.npy",
        f"{embed_path}/X_profile_val.npy",
        f"{embed_path}/X_theme_val.npy",
        f"{embed_path}/y_val.npy",
        f"{embed_path}/X_profile_test.npy",
        f"{embed_path}/X_theme_test.npy",
        f"{embed_path}/y_test.npy"
    ]

    # If all files already exist, skip computation
    if all(os.path.exists(f) for f in files_needed):
        print(f"✅ Embeddings already exist in {embed_path}, nothing to do.")
        return None

    model = SentenceTransformer(embedding_model)

    def encode_and_save(df, split_name):
        # Encode both profile and theme columns into embeddings
        emb_profile = model.encode(df[profile_col].tolist(), show_progress_bar=True, normalize_embeddings=True)
        emb_theme   = model.encode(df[theme_col].tolist(), show_progress_bar=True, normalize_embeddings=True)
        y           = df[label_col].values

        # Save embeddings and labels as .npy files
        np.save(f"{embed_path}/X_profile_{split_name}.npy", emb_profile)
        np.save(f"{embed_path}/X_theme_{split_name}.npy", emb_theme)
        np.save(f"{embed_path}/y_{split_name}.npy", y)

    # ------------------------
    # 🚀 Training + Validation
    # ------------------------
    tracker_trainval = EmissionsTracker(
        output_dir=embed_path,
        save_to_file=True,
        project_name=f"train_val"
    )
    tracker_trainval.start()

    encode_and_save(train, "train")
    encode_and_save(val, "val")

    emissions_trainval = tracker_trainval.stop()
    print(f"🌱 {embedding_model} - Embedding emissions (train+val): {emissions_trainval:.6f} kg CO₂")

    # ------------------------
    # 🚀 Test
    # ------------------------
    tracker_test = EmissionsTracker(
        output_dir=embed_path,
        save_to_file=True,
        project_name=f"test"
    )
    tracker_test.start()

    encode_and_save(test, "test")

    emissions_test = tracker_test.stop()
    print(f"🌱 {embedding_model} - Embedding emissions (test): {emissions_test:.6f} kg CO₂")

    return {
        "emissions_trainval": emissions_trainval,
        "emissions_test": emissions_test
    }


def load_embeddings(embedding_model, split):
    """
    Load embeddings and labels saved as .npy for a given embedding_model and split.
    split: "train", "val", or "test"
    """
    embed_path = f"embeddings/{embedding_model.replace('/', '_')}"
    X_profile = np.load(f"{embed_path}/X_profile_{split}.npy")
    X_theme   = np.load(f"{embed_path}/X_theme_{split}.npy")
    y         = np.load(f"{embed_path}/y_{split}.npy")
    return X_profile, X_theme, y


def combine_embeddings(X_profile, X_theme, method="concatenate"):
    """
    Combine profile and theme embeddings using different strategies:
      - concatenate: join embeddings side by side
      - mean: element-wise average
      - min: element-wise minimum
      - max: element-wise maximum
    """
    if method == "concatenate":
        return np.concatenate([X_profile, X_theme], axis=1)
    elif method == "mean":
        return (X_profile + X_theme) / 2
    elif method == "min":
        return np.minimum(X_profile, X_theme)
    elif method == "max":
        return np.maximum(X_profile, X_theme)
    else:
        raise ValueError(f"Unknown method: {method}")

In [25]:
embedding_models = [
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", 
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    "ibm-granite/granite-embedding-278m-multilingual"
]

# Iterate through the list of embedding models
for embedding_model in embedding_models:
    # Generate and save embeddings for train/val/test (only if they don't already exist)
    compute_and_save_embeddings(embedding_model, train, val, test)


🟢 Checking embeddings for sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...
Directory 'embeddings/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2' already exists.
✅ Embeddings already exist in embeddings/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2, nothing to do.

🟢 Checking embeddings for sentence-transformers/paraphrase-multilingual-mpnet-base-v2...
Directory 'embeddings/sentence-transformers_paraphrase-multilingual-mpnet-base-v2' already exists.
✅ Embeddings already exist in embeddings/sentence-transformers_paraphrase-multilingual-mpnet-base-v2, nothing to do.

🟢 Checking embeddings for ibm-granite/granite-embedding-278m-multilingual...
Directory 'embeddings/ibm-granite_granite-embedding-278m-multilingual' already exists.
✅ Embeddings already exist in embeddings/ibm-granite_granite-embedding-278m-multilingual, nothing to do.


## BERT Threshold

In [27]:
def run_threshold_pipeline(embedding_model, thresholds=range(5, 101, 5)):
    print(f"\n🟢 Threshold pipeline for {embedding_model}")
    base_path = "results/BERT_threshold"
    model_path = f"{base_path}/{embedding_model.replace('/', '_')}"
    create_directory(model_path)

    # Load embeddings (Xp = profile, Xt = theme, y = labels)
    Xp_train, Xt_train, y_train = load_embeddings(embedding_model, "train")
    Xp_val,   Xt_val,   y_val   = load_embeddings(embedding_model, "val")
    Xp_test,  Xt_test,  y_test  = load_embeddings(embedding_model, "test")

    # Merge train + val to find the best threshold
    Xp_train_val = np.vstack([Xp_train, Xp_val])
    Xt_train_val = np.vstack([Xt_train, Xt_val])
    y_train_val  = np.concatenate([y_train, y_val])

    results_summary = {}

    # ------------------------
    # 🚀 CodeCarbon emissions for training/validation (threshold search)
    # ------------------------
    tracker_train = EmissionsTracker(
        output_dir=model_path,
        save_to_file=True,
        project_name=f"train_val"
    )
    tracker_train.start()

    # Cosine similarity between embeddings
    sims_train_val = np.sum(Xp_train_val * Xt_train_val, axis=1) / (
        np.linalg.norm(Xp_train_val, axis=1) * np.linalg.norm(Xt_train_val, axis=1)
    )

    sims_train_val = (sims_train_val + 1) / 2

    best_threshold, best_f1 = None, 0
    results_train_val = {}

    # Search thresholds (5% to 100%)
    for t in thresholds:
        thr = t / 100  # scale to [0, 1]
        preds = (sims_train_val >= thr).astype(int)
        f1 = f1_score(y_train_val, preds, average="macro", zero_division=0)
        results_train_val[thr] = {"f1": f1}
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = thr

    emissions_train = tracker_train.stop()
    print(f"🌱 Training/validation emissions: {emissions_train:.6f} kg CO₂")
    print(f"⭐ Best threshold: {best_threshold} (F1={best_f1:.3f})")

    # Save train/val results
    save_dict_to_json(results_train_val, f"{model_path}/train_val.json")
    save_dict_to_json(
        {"best_threshold": best_threshold, "best_f1": best_f1, "emissions_train": emissions_train},
        f"{model_path}/best_threshold.json"
    )

    # ------------------------
    # 🚀 CodeCarbon emissions for test (using best threshold)
    # ------------------------
    tracker_test = EmissionsTracker(
        output_dir=model_path,
        save_to_file=True,
        project_name=f"test"
    )
    tracker_test.start()
    
    sims_test = np.sum(Xp_test * Xt_test, axis=1) / (
        np.linalg.norm(Xp_test, axis=1) * np.linalg.norm(Xt_test, axis=1)
    )

    sims_test = (sims_test + 1) / 2

    preds_test = (sims_test >= best_threshold).astype(int)
    report = classification_report(y_test, preds_test, output_dict=True)

    emissions_test = tracker_test.stop()
    print(f"🌱 Test emissions: {emissions_test:.6f} kg CO₂")

    # Save test results
    results_test = {
        "best_threshold": best_threshold,
        "classification_report": report,
        "emissions_train": emissions_train,
        "emissions_test": emissions_test
    }
    save_dict_to_json(results_test, f"{model_path}/test_report.json")
    save_dict_to_json(report, f"{model_path}/test_classification_report.json")
    

    results_summary["cosine_threshold"] = results_test
    return results_summary

In [28]:
for embedding_model in embedding_models:
    run_threshold_pipeline(embedding_model)


🟢 Threshold pipeline for sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Directory 'results/BERT_threshold/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2' created successfully!
🌱 Training/validation emissions: 0.000002 kg CO₂
⭐ Best threshold: 0.8 (F1=0.547)
✅ Dictionary saved at: results/BERT_threshold/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/train_val.json
✅ Dictionary saved at: results/BERT_threshold/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/best_threshold.json
🌱 Test emissions: 0.000001 kg CO₂
✅ Dictionary saved at: results/BERT_threshold/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/test_report.json
✅ Dictionary saved at: results/BERT_threshold/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/test_classification_report.json

🟢 Threshold pipeline for sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Directory 'results/BERT_threshold/sentence-transformers_paraphrase-multilingual-mpnet-b

## BERT Classifiers

In [15]:
# ------------------------
# Settings
# ------------------------
n_trials = 20
base_path = "results/BERT_classifiers"
create_directory(base_path)

# ------------------------
# Generic evaluation function
# ------------------------
def evaluate_and_return(trial, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
    rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

    # Save metrics in Optuna trial attributes for later analysis
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("precision", prec)
    trial.set_user_attr("recall", rec)
    trial.set_user_attr("f1", f1)

    return f1  # optimize based on F1-score

# ------------------------
# Objective functions for each classifier
# ------------------------
def make_objectives(X_train, y_train, X_val, y_val):
    def objective_knn(trial):
        n_neighbors = trial.suggest_int("n_neighbors", 2, 200)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        metric = trial.suggest_categorical("metric", ["euclidean", "manhattan", "cosine"])
        model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return evaluate_and_return(trial, y_val, y_pred)

    def objective_logreg(trial):
        C = trial.suggest_loguniform("C", 1e-3, 1e2)
        model = LogisticRegression(C=C, max_iter=2000, solver="lbfgs", random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return evaluate_and_return(trial, y_val, y_pred)

    def objective_dtree(trial):
        max_depth = trial.suggest_int("max_depth", 2, 50)
        criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        model = DecisionTreeClassifier(
            max_depth=max_depth, 
            criterion=criterion, 
            min_samples_split=min_samples_split, 
            random_state=random_state
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return evaluate_and_return(trial, y_val, y_pred)
    
    def objective_rf(trial):
        n_estimators = trial.suggest_int("n_estimators", 20, 300)
        criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        max_depth = trial.suggest_int("max_depth", 2, 20)
        model = RandomForestClassifier(
            n_estimators=n_estimators, 
            max_depth=max_depth,
            criterion=criterion,
            min_samples_split=min_samples_split,
            random_state=random_state
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return evaluate_and_return(trial, y_val, y_pred)
    
    def objective_svm(trial):
        C = trial.suggest_loguniform("C", 1e-3, 1e2)
        penalty = trial.suggest_categorical("penalty", ["l1", "l2"])
        model = LinearSVC(C=C, penalty=penalty, random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return evaluate_and_return(trial, y_val, y_pred)

    return {
        "KNN": objective_knn,
        "LogReg": objective_logreg,
        "DecisionTree": objective_dtree,
        "RandomForest": objective_rf,
        "SVM": objective_svm
    }

Directory 'results/BERT_classifiers' created successfully!


In [16]:
# ------------------------
# Main pipeline per embedding model
# ------------------------
def run_pipeline_for_embedding(embedding_model, methods=["concatenate", "mean", "min", "max"]):
    print(f"\n🟢 Running pipeline for embedding: {embedding_model}")
    embed_path = f"{base_path}/{embedding_model.replace('/', '_')}"
    create_directory(embed_path)

    # Load embeddings for train, val, test
    Xp_train, Xt_train, y_train = load_embeddings(embedding_model, "train")
    Xp_val,   Xt_val,   y_val   = load_embeddings(embedding_model, "val")
    Xp_test,  Xt_test,  y_test  = load_embeddings(embedding_model, "test")

    # Loop over embedding combination methods
    for method in methods:
        print(f"\n🔀 Testing combination method: {method}")
        method_path = f"{embed_path}/{method}"
        create_directory(method_path)

        # Combine embeddings (profile + theme)
        X_train = combine_embeddings(Xp_train, Xt_train, method=method)
        X_val   = combine_embeddings(Xp_val,   Xt_val,   method=method)
        X_test  = combine_embeddings(Xp_test,  Xt_test,  method=method)

        # Merge train+val for final training after hyperparameter tuning
        X_train_val = np.vstack([X_train, X_val])
        y_train_val = np.concatenate([y_train, y_val])

        # Define objective functions for each classifier
        objectives = make_objectives(X_train, y_train, X_val, y_val)

        # Loop through classifiers
        for name, obj in objectives.items():
            print(f"\n🔎 Running Optuna study for {name} ({method})...")
            model_path = f"{method_path}/{name}"
            create_directory(model_path)

            # 🚀 Track emissions during training/validation
            tracker_train = EmissionsTracker(
                output_dir=model_path, save_to_file=True, 
                project_name=f"train_val"
            )
            tracker_train.start()

            sampler = TPESampler(seed=random_state)
            study = optuna.create_study(direction="maximize", sampler=sampler)
            study.optimize(obj, n_trials=n_trials)

            emissions_train = tracker_train.stop()
            print(f"🌱 {name} ({method}) - Train/Validation emissions: {emissions_train:.6f} kg CO₂")

            # Save all trial results
            trials_summary = [
                {
                    "number": t.number,
                    "params": t.params,
                    "accuracy": t.user_attrs.get("accuracy"),
                    "precision": t.user_attrs.get("precision"),
                    "recall": t.user_attrs.get("recall"),
                    "f1": t.user_attrs.get("f1"),
                    "method": method   # log combination method used
                }
                for t in study.trials
            ]
            save_dict_to_json(trials_summary, f"{model_path}/train_val.json")

            # ------------------------
            # Evaluate best trial on test set
            # ------------------------
            best_trial = study.best_trial
            params = best_trial.params

            if name == "KNN":
                model = KNeighborsClassifier(**params)
            elif name == "LogReg":
                model = LogisticRegression(max_iter=2000, solver="lbfgs", random_state=random_state, **params)
            elif name == "DecisionTree":
                model = DecisionTreeClassifier(random_state=random_state, **params)
            elif name == "RandomForest":
                model = RandomForestClassifier(random_state=random_state, **params)
            elif name == "SVM":
                model = LinearSVC(random_state=random_state, **params)

            # Retrain with train+val data
            model.fit(X_train_val, y_train_val)

            # 🚀 Track emissions during test evaluation
            tracker_test = EmissionsTracker(
                output_dir=model_path, save_to_file=True, 
                project_name=f"test"
            )
            tracker_test.start()
            y_pred_test = model.predict(X_test)
            emissions_test = tracker_test.stop()

            print(f"🌱 {name} ({method}) - Test emissions: {emissions_test:.6f} kg CO₂")

            # Compute final test metrics
            report = classification_report(y_test, y_pred_test, output_dict=True)

            results_test = {
                "best_params": params,
                "classification_report": report,
                "emissions_train": emissions_train,
                "emissions_test": emissions_test,
                "method": method
            }
            save_dict_to_json(results_test, f"{model_path}/test_results.json")
            save_dict_to_json(report, f"{model_path}/test_classification_report.json")


In [17]:
for emb in embedding_models:
    run_pipeline_for_embedding(emb)


🟢 Running pipeline for embedding: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Directory 'results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2' created successfully!

🔀 Testing combination method: concatenate
Directory 'results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate' created successfully!

🔎 Running Optuna study for KNN (concatenate)...
Directory 'results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/KNN' created successfully!


[I 2025-08-26 17:58:27,730] A new study created in memory with name: no-name-a15add4a-98b2-439a-a7ca-2287ca0844a5
[I 2025-08-26 17:58:29,245] Trial 0 finished with value: 0.5875880142660859 and parameters: {'n_neighbors': 76, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 0 with value: 0.5875880142660859.
[I 2025-08-26 17:58:32,507] Trial 1 finished with value: 0.649887088143336 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'metric': 'cosine'}. Best is trial 1 with value: 0.649887088143336.
[I 2025-08-26 17:58:34,381] Trial 2 finished with value: 0.5674682075575262 and parameters: {'n_neighbors': 167, 'weights': 'uniform', 'metric': 'cosine'}. Best is trial 1 with value: 0.649887088143336.
[I 2025-08-26 17:58:35,846] Trial 3 finished with value: 0.7337167218897807 and parameters: {'n_neighbors': 87, 'weights': 'distance', 'metric': 'cosine'}. Best is trial 3 with value: 0.7337167218897807.
[I 2025-08-26 17:58:50,856] Trial 4 finished with value: 0.5784368620440

🌱 KNN (concatenate) - Train/Validation emissions: 0.000094 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/KNN/train_val.json
🌱 KNN (concatenate) - Test emissions: 0.000004 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/KNN/test_results.json
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/KNN/test_classification_report.json

🔎 Running Optuna study for LogReg (concatenate)...
Directory 'results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/LogReg' created successfully!


[I 2025-08-26 18:00:11,526] A new study created in memory with name: no-name-b7b4f48d-59ec-4deb-9f14-b88eb5203d9c
/tmp/ipykernel_68979/3039998044.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e2)
[I 2025-08-26 18:00:12,035] Trial 0 finished with value: 0.48797550061203876 and parameters: {'C': 0.0745934328572655}. Best is trial 0 with value: 0.48797550061203876.
/tmp/ipykernel_68979/3039998044.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e2)
[I 2025-08-26 18:00:16,625] Trial 1 finished with value: 0.6002601981006912 and parameters: {'C': 56.69849511478853}. Best is trial 1 w

🌱 LogReg (concatenate) - Train/Validation emissions: 0.000092 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/LogReg/train_val.json
🌱 LogReg (concatenate) - Test emissions: 0.000002 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/LogReg/test_results.json
✅ Dictionary saved at: results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/LogReg/test_classification_report.json

🔎 Running Optuna study for DecisionTree (concatenate)...
Directory 'results/BERT_classifiers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/concatenate/DecisionTree' created successfully!


...

🌱 DecisionTree (max) - Train/Validation emissions: 0.000308 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/DecisionTree/train_val.json
🌱 DecisionTree (max) - Test emissions: 0.000001 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/DecisionTree/test_results.json
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/DecisionTree/test_classification_report.json

🔎 Running Optuna study for RandomForest (max)...
Directory 'results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/RandomForest' created successfully!


[I 2025-08-27 02:45:15,820] A new study created in memory with name: no-name-06129c4b-29a9-447a-94c5-a4ed2f5e34f9
[I 2025-08-27 02:45:34,451] Trial 0 finished with value: 0.47540691978443844 and parameters: {'n_estimators': 125, 'criterion': 'gini', 'min_samples_split': 4, 'max_depth': 4}. Best is trial 0 with value: 0.47540691978443844.
[I 2025-08-27 02:45:55,173] Trial 1 finished with value: 0.7174070811069685 and parameters: {'n_estimators': 36, 'criterion': 'gini', 'min_samples_split': 2, 'max_depth': 20}. Best is trial 1 with value: 0.7174070811069685.
[I 2025-08-27 02:47:28,132] Trial 2 finished with value: 0.5843549790397848 and parameters: {'n_estimators': 253, 'criterion': 'gini', 'min_samples_split': 7, 'max_depth': 11}. Best is trial 1 with value: 0.7174070811069685.
[I 2025-08-27 02:48:10,500] Trial 3 finished with value: 0.5310834524275406 and parameters: {'n_estimators': 141, 'criterion': 'entropy', 'min_samples_split': 7, 'max_depth': 8}. Best is trial 1 with value: 0.71

🌱 RandomForest (max) - Train/Validation emissions: 0.001161 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/RandomForest/train_val.json
🌱 RandomForest (max) - Test emissions: 0.000000 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/RandomForest/test_results.json
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/RandomForest/test_classification_report.json

🔎 Running Optuna study for SVM (max)...
Directory 'results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/SVM' created successfully!


[I 2025-08-27 03:14:28,167] A new study created in memory with name: no-name-0c1ff43b-f56c-4f2c-b0cd-1775e6002f81
/tmp/ipykernel_68979/3039998044.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 1e2)
/home/matheus/Desktop/Itens./Itens/Projetos/LeanDL-HPC-BERT/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
[I 2025-08-27 03:14:34,720] Trial 0 finished with value: 0.5169983445069235 and parameters: {'C': 0.0745934328572655, 'penalty': 'l1'}. Best is trial 0 with value: 0.5169983445069235.
/tmp/ipykernel_68979/3039998044.py:76: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releas

🌱 SVM (max) - Train/Validation emissions: 0.000011 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/SVM/train_val.json
🌱 SVM (max) - Test emissions: 0.000001 kg CO₂
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/SVM/test_results.json
✅ Dictionary saved at: results/BERT_classifiers/ibm-granite_granite-embedding-278m-multilingual/max/SVM/test_classification_report.json
